In [1]:
import numpy as np
import scanpy as sc
import h5py
import os
from scbig.utils import setup_seed,louvain,calculate_metric
import warnings
warnings.filterwarnings('ignore')
import simba as si
import shutil

Using backend: pytorch


In [2]:
#shutil.rmtree('result_simba_rnaseq')  

In [3]:
workdir = 'result_simba_rnaseq'
si.settings.set_workdir(workdir)
si.settings.set_figure_params(dpi=80,style='white',fig_size=[5,5],rc={'image.cmap': 'viridis'})

Saving results in: result_simba_rnaseq


In [4]:
setup_seed(0)

In [5]:
dataset = 'YAN'
#dataset = 'Deng'
#dataset = 'DPT'
method = 'SIMBA'
dir0 = '../'
dir1 = '{}'.format(dataset)

with h5py.File(os.path.join(dir0, 'datasets/trajectory/{}.h5'.format(dataset))) as data_mat:
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    X = np.ceil(X).astype(np.int_)
    Y = np.array(Y).astype(np.int_).squeeze()

adata = sc.AnnData(X)
adata.obs['cl_type'] = Y
n_clusters = len(np.unique(Y))
print(adata)

AnnData object with n_obs × n_vars = 90 × 20214
    obs: 'cl_type'


In [6]:
si.pp.filter_genes(adata, min_n_cells=3)
si.pp.filter_cells_rna(adata, min_n_genes=100)
si.pp.normalize(adata, method='lib_size')
si.pp.log_transform(adata)
print(adata)

Before filtering: 
90 cells, 20214 genes
Filter genes based on min_n_cells
After filtering out low-expressed genes: 
90 cells, 18042 genes
before filtering: 
90 cells,  18042 genes
filter cells based on min_n_genes
after filtering out low-quality cells: 
90 cells,  18042 genes
AnnData object with n_obs × n_vars = 90 × 18042
    obs: 'cl_type', 'n_counts', 'n_genes', 'pct_genes'
    var: 'n_counts', 'n_cells', 'pct_cells'
    layers: 'raw'


In [7]:
si.tl.discretize(adata, n_bins=5)
si.tl.gen_graph(list_CG=[adata],use_highly_variable=False,dirname='graph0')

relation0: source: C, destination: G
#edges: 760341
relation1: source: C, destination: G
#edges: 151390
relation2: source: C, destination: G
#edges: 51339
relation3: source: C, destination: G
#edges: 17861
relation4: source: C, destination: G
#edges: 7420
Total number of edges: 988351
Writing graph file "pbg_graph.txt" to "result_simba_rnaseq\pbg\graph0" ...
Finished.


In [8]:
dict_config = si.settings.pbg_params.copy()
dict_config['dimension'] = 64
si.tl.pbg_train(pbg_params=dict_config, auto_wd=True, save_wd=True, output='model')

Auto-estimated weight decay is 0.035853
`.settings.pbg_params['wd']` has been updated to 0.035853
Converting input data ...
[2023-08-20 14:01:13.950062] Using the 5 relation types given in the config
[2023-08-20 14:01:13.950062] Searching for the entities in the edge files...
[2023-08-20 14:01:15.162338] Entity type C:
[2023-08-20 14:01:15.163338] - Found 90 entities
[2023-08-20 14:01:15.163338] - Removing the ones with fewer than 1 occurrences...
[2023-08-20 14:01:15.164338] - Left with 90 entities
[2023-08-20 14:01:15.164338] - Shuffling them...
[2023-08-20 14:01:15.164338] Entity type G:
[2023-08-20 14:01:15.165338] - Found 18042 entities
[2023-08-20 14:01:15.165338] - Removing the ones with fewer than 1 occurrences...
[2023-08-20 14:01:15.168340] - Left with 18042 entities
[2023-08-20 14:01:15.168340] - Shuffling them...
[2023-08-20 14:01:15.176341] Preparing counts and dictionaries for entities and relation types:
[2023-08-20 14:01:15.177341] - Writing count of entity type C and p

In [9]:
dict_adata = si.read_embedding()
print(dict_adata)

{'C': AnnData object with n_obs × n_vars = 90 × 64, 'G': AnnData object with n_obs × n_vars = 18042 × 64}


In [10]:
adata_C = dict_adata['C']
adata_C.obs_names = adata_C.obs_names.astype('str')
adata.obsm['feat'] = np.array(adata_C[adata.obs_names,:].X)
adata.obsm['feat'].shape

(90, 64)

In [11]:
 # louvain
adata = louvain(adata, resolution=1, use_rep='feat')
y_pred_l = np.array(adata.obs['louvain'])
n_pred = len(np.unique(y_pred_l))
nmi_l, ari_l = np.round(calculate_metric(Y, y_pred_l), 4)
print('Clustering Louvain: NMI= %.4f, ARI= %.4f' % (nmi_l, ari_l))

Clustering Louvain: NMI= 0.8322, ARI= 0.8057


In [12]:
sc.tl.umap(adata)
print(adata)

np.savez(os.path.join(dir0, "results/trajectory_inference/{}/{}_{}.npz".format(dataset, dataset, method)),
         true=Y,
         umap=adata.obsm['X_umap'],
         latent=adata.obsm['feat'],
         data=np.array(adata.X.todense()),
         louvain=np.array(adata.obs['louvain'].values.astype(int)))

AnnData object with n_obs × n_vars = 90 × 18042
    obs: 'cl_type', 'n_counts', 'n_genes', 'pct_genes', 'pbg_id', 'louvain'
    var: 'n_counts', 'n_cells', 'pct_cells', 'pbg_id'
    uns: 'disc', 'neighbors', 'louvain', 'umap'
    obsm: 'feat', 'X_umap'
    layers: 'raw', 'disc'
    obsp: 'distances', 'connectivities'
